In [2]:
import numpy as np
import kimimaro
from skimage import io
import file_utilities as fu
import napari

# Get the file path
file_path = fu.choose_file()

# Create an instance of LifImageProcessor
# 加载 TIF 格式的图像文件
image = io.imread(file_path)

viewer = napari.Viewer()

viewer.add_labels(image, name='The label Result', blending='additive')


Selected File: C:/Users/Ding-Yang/Desktop/internship/Try don't merge/better images/Unet/Preprocess image/20220815 necroptosis Paula_PS-A2-60 series 53/GT.tif


<Labels layer 'The label Result' at 0x28c7b887d10>

# Generate the border everywhere

In [14]:
import numpy as np
import scipy.ndimage as ndi

import numpy as np

def make_border_between_touched_label(input_image, change_yz = False):
    if change_yz:
        image_flip = np.swapaxes(input_image, 0, 2)
    else:
        image_flip = np.copy(input_image)
    border_mask = image_flip > 0
    x, y, z = image_flip.shape
    for a in range(0, x):
        for b in range(0, y-1, 1):
            for c in range(0, z-1, 1):
                image_slice = image_flip[a]
                piece = np.copy(image_slice[b:b+2, c:c+2])
                def find_unique_intensities(image):
                    # 获取图像中除了零以外的唯一强度值
                    unique_values = np.unique(image)
                    
                    # 移除零以外的唯一值
                    unique_values = unique_values[unique_values != 0]
                     
                    # 检查除了零以外的唯一值数量是否大于 1
                    if len(unique_values) > 1:
                        # 返回除了零以外的唯一值元组
                        return True # 转换为元组
                
                unique_intensities = find_unique_intensities(piece)
                
                # 将唯一值元组添加到集合中
                if unique_intensities:
                    image_flip[a, b:b+2, c:c+2] = 0

    # 将集合转换为列表形式，并返回结果
    result = image_flip > 0
    border_mask = border_mask ^ result
    if change_yz:
        image_flip = np.swapaxes(image_flip, 2, 0)
        border_mask = np.swapaxes(border_mask, 2, 0)

    return border_mask #image_flip

def generate_border_3d(image, border_size=1):
    # Initialize an empty array to store the eroded slices
    eroded_slices = []

    xy_border_mask = make_border_between_touched_label(image)
    yz_border_mask = make_border_between_touched_label(image,change_yz = True)
    
    eroded_volume = np.zeros_like(image, dtype=np.uint8)
    eroded_volume[xy_border_mask] = 127
    eroded_volume[yz_border_mask] = 127
    # Apply dilation to the eroded volume to create the border
    kernel_size = 2 * border_size + 1
    dilation_kernel = np.ones((kernel_size, kernel_size, kernel_size))  # Kernel for dilation
    border_volume = ndi.binary_dilation(image, structure=dilation_kernel, iterations=1)

    # Subtract the eroded volume from the original image to obtain the border
    border_volume = np.where(image > 0, 0 , border_volume).astype(np.uint8)
    border_volume = border_volume +eroded_volume
    ## Replace 255 values to 127 for all pixels. Eventually we will only define border pixels with this value
    original_with_border = np.where(border_volume > 0, 127 , border_volume).astype(np.uint8)
    #original_with_border[border_mask] = 127
    image_255 = np.where(image > 0, 255 , image).astype(np.uint8)
    original_with_border = original_with_border + image_255
    original_with_border = np.where((original_with_border < 255) & (original_with_border > 0), 127, original_with_border).astype(np.uint8)
    # Extract border and labeled regions using boolean masks
    border = np.where(original_with_border == 255, 0 , original_with_border).astype(np.uint8)
    border = np.where(border != 0, 2 , border).astype(np.uint8)

    labeled = np.where(original_with_border == 127, 0 , original_with_border).astype(np.uint8)
    labeled = np.where(labeled != 0, 1 , labeled).astype(np.uint8)
    label_wih_border = np.zeros_like(image, dtype=np.uint8)
    label_wih_border = labeled + border
    return border, labeled, label_wih_border
    

In [29]:
label_wih_border.shape

(17, 1024, 1024)

In [30]:
#correct_segement_bm4d_labels = viewer.layers['verion_3'].data

border, labeled, label_wih_border = generate_border_3d(image)
viewer.add_labels(border, name='border', blending='additive')
viewer.add_labels(labeled, name='labeled', blending='additive')
viewer.add_labels(label_wih_border, name='label_wih_border', blending='additive')

<Labels layer 'label_wih_border' at 0x28d90cda290>

In [12]:
import numpy as np
import tifffile
import os

def save_in_same_directory(original_file_path, file_name):
    directory = os.path.dirname(original_file_path)
    save_path = os.path.join(directory, file_name)
    return save_path
# Generate some random 3D data

# Save the 3D data as a TIFF file
tifffile.imwrite(save_in_same_directory(file_path, 'label_wih_border_mask.tif'), label_wih_border)
tifffile.imwrite(save_in_same_directory(file_path,'border_mask.tif'), border)
tifffile.imwrite(save_in_same_directory(file_path,'labeled_mask.tif'), labeled)

In [11]:
label_wih_border = viewer.layers['label_wih_border'].data

tifffile.imwrite("C:/Users/Ding-Yang/Desktop/internship/Try don't merge/better images/Unet/GT/section 2 740 TexRed Dextrans 3K dil 2 30 min CertnL 11.05.23/border.tif", border)


In [6]:
save_in_same_directory(file_path, 'label_wih_border_mask')

"C:/Users/Ding-Yang/Desktop/internship/Try don't merge/better images/Unet/GT/section 2 740 TexRed Dextrans 3K dil 2 30 min CertnL 11.05.23\\label_wih_border_mask"

# Generate the border between two connecting labels

In [3]:
#correct_segement_bm4d_labels = viewer.layers['verion_3'].data

border, labeled, label_wih_border = generate_border_3d(image)
viewer.add_labels(border, name='border', blending='additive')
viewer.add_labels(labeled, name='labeled', blending='additive')
viewer.add_labels(label_wih_border, name='label_wih_border', blending='additive')

<Labels layer 'label_wih_border' at 0x1b39911bb10>

In [4]:
import numpy as np
import tifffile
import os

def save_in_same_directory(original_file_path, file_name):
    directory = os.path.dirname(original_file_path)
    save_path = os.path.join(directory, file_name)
    return save_path
# Generate some random 3D data

# Save the 3D data as a TIFF file
tifffile.imwrite(save_in_same_directory(file_path, 'label_wih_border_mask_2.tif'), label_wih_border)
tifffile.imwrite(save_in_same_directory(file_path,'border_mask_2.tif'), border)
tifffile.imwrite(save_in_same_directory(file_path,'labeled_mask_2.tif'), labeled)

C:\Users\Ding-Yang\anaconda3\envs\intern\Lib\site-packages\napari\components\viewer_model.py:1181: RuntimeWarning: Can't find napari-aicsimageio plugin associated with C:\Users\Ding-Yang\Desktop\internship\Try don't merge\better images\Unet\GT\section 2 740 TexRed Dextrans 3K dil 2 30 min CertnL 11.05.23\original_label.tif files. This may be because you've switched environments, or have uninstalled the plugin without updating the reader preference. You can remove this preference in the preference dialog, or by editing `settings.plugins.extension2reader`.
  warnings.warn(
C:\Users\Ding-Yang\anaconda3\envs\intern\Lib\site-packages\napari\components\viewer_model.py:1181: RuntimeWarning: Can't find napari-aicsimageio plugin associated with C:\Users\Ding-Yang\Desktop\internship\Try don't merge\better images\Unet\GT\section 2 740 TexRed Dextrans 3K dil 2 30 min CertnL 11.05.23\verion_2.tif files. This may be because you've switched environments, or have uninstalled the plugin without updatin

# Generate border base on yz

In [7]:
import numpy as np
import scipy.ndimage as ndi

import numpy as np
def make_border_between_touched_label(input_image, change_yz = False):
    if change_yz:
        image_flip = np.swapaxes(input_image, 0, 2)
    else:
        image_flip = np.copy(input_image)
    border_mask = image_flip > 0
    x, y, z = image_flip.shape
    for a in range(0, x):
        for b in range(0, y-1, 1):
            for c in range(0, z-1, 1):
                image_slice = image_flip[a]
                piece = np.copy(image_slice[b:b+2, c:c+2])
                def find_unique_intensities(image):
                    # 获取图像中除了零以外的唯一强度值
                    unique_values = np.unique(image)
                    
                    # 移除零以外的唯一值
                    unique_values = unique_values[unique_values != 0]
                     
                    # 检查除了零以外的唯一值数量是否大于 1
                    if len(unique_values) > 1:
                        # 返回除了零以外的唯一值元组
                        return True # 转换为元组
                
                unique_intensities = find_unique_intensities(piece)
                
                # 将唯一值元组添加到集合中
                if unique_intensities:
                    image_flip[a, b:b+2, c:c+2] = 0

    # 将集合转换为列表形式，并返回结果
    result = image_flip > 0
    border_mask = border_mask ^ result
    if change_yz:
        image_flip = np.swapaxes(image_flip, 2, 0)
        border_mask = np.swapaxes(border_mask, 2, 0)

    return border_mask #image_flip
    
def generate_border_3d(image, border_size=1):
    # Initialize an empty array to store the eroded slices
    image_flip = np.swapaxes(image, 0, 2)

    eroded_slices = []

    # Iterate over each slice in the 3D image
    for slice_idx in range(image_flip.shape[0]):
        # Perform erosion on the current slice
   #     eroded_slice = image_flip[slice_idx] * (np.abs(ndi.laplace(image_flip[slice_idx])) > 0)
        eroded_slice = ndi.binary_dilation(image_flip[slice_idx], iterations=1)

        # Append the eroded slice to the list
        eroded_slices.append(eroded_slice)

    # Stack the eroded slices along the z-axis to create a 3D array
    eroded_volume = np.stack(eroded_slices, axis=0)
    eroded_image = np.swapaxes(eroded_volume, 2, 0)
    eroded_image = np.where(eroded_image > 0, 255 , eroded_image).astype(np.uint8)


    #original_with_border[border_mask] = 127
    image_255 = np.where(image > 0, 255 , image).astype(np.uint8)
    border = eroded_image - image_255
    yz_border_mask = make_border_between_touched_label(image,change_yz = True)
    
    border[yz_border_mask] = 255
    border = np.where(border != 0, 2 , border).astype(np.uint8)

    labeled = np.where((border != 2) & (image_255 > 0), 1, image).astype(np.uint8)

    label_wih_border = np.zeros_like(image, dtype=np.uint8)
    label_wih_border = labeled + border
    return border, labeled, label_wih_border


    

In [8]:
#correct_segement_bm4d_labels = viewer.layers['verion_3'].data

border, labeled, label_wih_border = generate_border_3d(image)
viewer.add_labels(border, name='border', blending='additive')
viewer.add_labels(labeled, name='labeled', blending='additive')
viewer.add_labels(label_wih_border, name='label_wih_border', blending='additive')

<Labels layer 'label_wih_border' at 0x28c12f83550>

In [9]:
import numpy as np
import tifffile
import os

def save_in_same_directory(original_file_path, file_name):
    directory = os.path.dirname(original_file_path)
    save_path = os.path.join(directory, file_name)
    print(save_path)
    return save_path
# Generate some random 3D data

# Save the 3D data as a TIFF file
tifffile.imwrite(save_in_same_directory(file_path, 'label_wih_border_mask_3.tif'), label_wih_border)
tifffile.imwrite(save_in_same_directory(file_path,'border_mask_3.tif'), border)
tifffile.imwrite(save_in_same_directory(file_path,'labeled_mask_3.tif'), labeled)

C:/Users/Ding-Yang/Desktop/internship/Try don't merge/better images/Unet/Preprocess image/20220815 necroptosis Paula_PS-A2-60 series 53\label_wih_border_mask_3.tif
C:/Users/Ding-Yang/Desktop/internship/Try don't merge/better images/Unet/Preprocess image/20220815 necroptosis Paula_PS-A2-60 series 53\border_mask_3.tif
C:/Users/Ding-Yang/Desktop/internship/Try don't merge/better images/Unet/Preprocess image/20220815 necroptosis Paula_PS-A2-60 series 53\labeled_mask_3.tif


C:\Users\Ding-Yang\anaconda3\envs\intern\Lib\site-packages\napari\components\viewer_model.py:1181: RuntimeWarning: Can't find napari-aicsimageio plugin associated with C:\Users\Ding-Yang\Desktop\internship\Try don't merge\better images\Unet\Preprocess image\20220815 necroptosis Paula_PS-A2-60 series 53\label_wih_border_mask_3.tif files. This may be because you've switched environments, or have uninstalled the plugin without updating the reader preference. You can remove this preference in the preference dialog, or by editing `settings.plugins.extension2reader`.
  warnings.warn(
